Lets create a Word Count Application. We will write code to calculate the most common words in [A Tale of Two Cities by Charles Dickens](https://www.gutenberg.org/ebooks/98) from Project Gutenberg.   
Before we read the data, we are going to create a sample data frame and write functions that count words, remove punctuations. We would test those functions and once everything works fine we read   
the data and create a full application.

Lets create a DataFrame by using a Python list of tuples.   
Then we print the data and schema. But before all of this, we will check spark version and type of sqlContext.

In [3]:
print sc.version
print type(sqlContext)

In [4]:
dataDF = sqlContext.createDataFrame([('Jax',), ('Rammus',), ('Zac',), ('Xin', ), ('Hecarim', ), ('Zac', ), ('Rammus', )], ['Jungler'])
dataDF.show()
print type(dataDF)
dataDF.printSchema()

In [5]:
# lets create a new DataFrame by adding a 'y' to each word.
# to concatenate we would be using concat and lit function.
from pyspark.sql.functions import lit, concat

newDataDF = dataDF.select(concat(dataDF.Jungler, lit('y')).alias('jungler'))
newDataDF.show()

In [6]:
# lets find the number of characters in each word.
from pyspark.sql.functions import length
newDataLengthDF = newDataDF.select(length('jungler'))
newDataLengthDF.show()

Lets perform aggregations by grouping data on the DataFrame by using groupBy function.   
Using groupBy lets find the counts of words.

In [8]:
dataCountsDF = (dataDF
                .groupBy('Jungler')
                .count()
                )
dataCountsDF.show()

Lets define a function for word counting. Lets use the technique that we implemented earlier.

In [10]:
def wordCount(wordsDF):
  """ Creates a DataFrame with word counts.
  
  Args:
      wordsDF: A DataFrame consisting of one string column called 'Jungler'.
      
  Returns:
      DataFrame of a (str, int): containing Jungler and count columns.
  """
  return (wordsDF
           .groupBy('Jungler')
           .count())

wordCount(dataDF).show()

If we have a large notebook and it is getting difficult to find the DataFrame and their schema, Spark provides a very useful function printDataFrames to find   
all the DataFrame created so far.

In [12]:
from spark_notebook_helpers import printDataFrames
printDataFrames(True)

In [13]:
# calculate number of unique junglers
uniqueJunglers = wordCount(dataDF).count()
print uniqueJunglers

# calculate mean number of occurences
averageCount = round(((dataCountsDF.groupBy().mean('count').collect())[0][0]), 2)
print averageCount

Lets do some data wrangling operation. As we are going to work on dataset of text files, we would keep the following assumptions:
+ words should be counted independent of their capitalization.
+ punctuation should be removed.
+ leading and trailing spaces should be removed.

To keep the above assumptions intact, we create a function called removePunctuation to perform the three operations.

In [15]:
from pyspark.sql.functions import regexp_replace, trim, col, lower
def removePunctuation(column):
  """Removes punctuation, changes to lower case, strips leading and trailing spaces.
  
  Args:
    column: a sentence
  
  Returns:
    Column: a column named 'sentence'
  """
  
  return lower(trim(regexp_replace(column, '[^\w\s]+', ""))).alias('sentence')

In [16]:
# test removePunctuation operation
sentenceDF = sqlContext.createDataFrame([('Hello, World!', ), (' How about some ice-cream?',), ('  *  Well, how about a cookie and lots of candies? *',)], ['sentence'])
sentenceDF.show(truncate=False)
(sentenceDF
  .select(removePunctuation(col('sentence')))
  .show(truncate=False))

We have both the functions ready, lets read the dataset and perform the operations accordingly.

In [18]:
filename = "/FileStore/tables/tux1rb3k1481050985024/TaleOfTwoCities.txt"
twoCitiesDF = sqlContext.read.text(filename).select(removePunctuation(col('value')))
twoCitiesDF.show(truncate=False)

Before we use the wordCount() function, we have to address the following issues:
+ Need to split each line by spaces.
+ Filter out empty lines or words.   
Lets address these two issues first.

In [20]:
from pyspark.sql.functions import split,explode
twoCitiesWordsDF = (twoCitiesDF
                    .select(explode(split(twoCitiesDF.sentence, " ")).alias('word'))
                    .where("word != ''")
                    )
twoCitiesWordsDF.show(truncate=False)
twoCitiesWordsDFCount = twoCitiesWordsDF.count()
print twoCitiesWordsDFCount

One more issue to address. We need to refactor wordCount() in order to groupby word

In [22]:
def wordCount(wordsDF):
  """ Creates a DataFrame with word counts.
  
  Args:
      wordsDF: A DataFrame consisting of one string column called 'Jungler'.
      
  Returns:
      DataFrame of a (str, int): containing Jungler and count columns.
  """
  return (wordsDF
           .groupBy('word')
           .count())

In [23]:
# produce a list of words in descending order.
from pyspark.sql.functions import desc
topWordsAndCountsDF = wordCount(twoCitiesWordsDF).orderBy(desc('count'))
topWordsAndCountsDF.show()

We see a lot of stop words like a, the, of etc in the top 20 words.  
We will keep those for now, will remove them in the next project.